In [1]:
%load_ext autoreload
%autoreload 2

In [21]:
import torch
from torch.optim import lr_scheduler
import torch.optim as optim
from torch.autograd import Variable

from trainer import fit
import numpy as np
cuda = torch.cuda.is_available()

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

In [2]:
from torchvision.datasets import ImageFolder

In [3]:
train_dataset = ImageFolder("/workspaces/google-kaggle-competition/data/teacher_data/Dishes")

In [4]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 9000
    Root location: /workspaces/google-kaggle-competition/data/teacher_data/Dishes

In [5]:
train_dataset[0]

(<PIL.Image.Image image mode=RGB size=224x224>, 0)

In [6]:
set(train_dataset.targets)

{0, 1, 2, 3, 4, 5}

In [7]:
train_dataset.class_to_idx

{'American': 0,
 'Chinese': 1,
 'European': 2,
 'Indian': 3,
 'Japanese': 4,
 'Korean': 5}

In [8]:
from datasets import TripletMNIST


In [9]:
triplet_train_dataset = TripletMNIST(train_dataset, True) # Returns triplets of images


In [10]:
import torch

In [30]:
batch_size = 128
kwargs = {'num_workers': 1, 'pin_memory': True} if torch.cuda.is_available() else {}
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)


In [12]:
cuda = torch.cuda.is_available()

In [13]:
import numpy as np

In [20]:
triplet_train_dataset = TripletMNIST(train_dataset, train=True) # Returns triplets of images
# triplet_test_dataset = TripletMNIST(test_dataset)
batch_size = 128
kwargs = {'num_workers': 1, 'pin_memory': True} if cuda else {}
triplet_train_loader = torch.utils.data.DataLoader(triplet_train_dataset, batch_size=batch_size, shuffle=True, **kwargs)
# triplet_test_loader = torch.utils.data.DataLoader(triplet_test_dataset, batch_size=batch_size, shuffle=False, **kwargs)

# Set up the network and training parameters
from networks import EmbeddingNet, TripletNet
from losses import TripletLoss

margin = 1.
embedding_net = EmbeddingNet()
model = TripletNet(embedding_net)

if cuda:
    model.cuda()
loss_fn = TripletLoss(margin)
lr = 1e-3
optimizer = optim.Adam(model.parameters(), lr=lr)
scheduler = lr_scheduler.StepLR(optimizer, 8, gamma=0.1, last_epoch=-1)
n_epochs = 1
log_interval = 100

In [26]:
for (data, target) in (triplet_train_loader):
   test = data
   break

In [34]:
train_dataset[0]

(<PIL.Image.Image image mode=RGB size=224x224>, 0)

In [24]:
fit(triplet_train_loader,  model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)

torch.Size([128])
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/root/.cache/pypoetry/virtualenvs/google-kaggle-TDiTHuYo-py3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_947/3520775775.py", line 1, in <cell line: 1>
    fit(triplet_train_loader,  model, loss_fn, optimizer, scheduler, n_epochs, cuda, log_interval)
  File "/workspaces/google-kaggle-competition/notebooks/trainer.py", line 23, in fit
    train_loss, metrics = train_epoch(train_loader, model, loss_fn, optimizer, cuda, log_interval, metrics)
  File "/workspaces/google-kaggle-competition/notebooks/trainer.py", line 60, in train_epoch
    raise KeyboardInterrupt
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/root/.cache/pypoetry/virtualenvs/google-kaggle-TDiTHuYo-py3.10/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 1993, 